In [ ]:
import os
from openai import OpenAI

token = os.environ["GITHUB_TOKEN"]
print(token)
print(os.environ["OPENAI_API_KEY"])
endpoint = "https://models.inference.ai.azure.com"
model_name = "gpt-4o-mini"

client = OpenAI(
    base_url=endpoint,
    api_key=token,
)

response = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant.",
        },
        {
            "role": "user",
            "content": "What is the capital of England?",
        }
    ],
    temperature=0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)


In [4]:
response.choices[0].message.content

'The capital of England is London.'

### Create QA app

In [5]:
from langchain.chains import RetrievalQA
from langchain_openai import OpenAI
from langchain.document_loaders import CSVLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch


In [27]:
file = "OutdoorClothingCatalog_1000_short.csv"
loader = CSVLoader(file_path=file,encoding='utf-8')


In [28]:
data = loader.load()

In [31]:
from langchain_openai import OpenAIEmbeddings

index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=OpenAIEmbeddings(model="text-embedding-3-large"),
).from_loaders([loader])

In [32]:
llm = OpenAI(model_name="gpt-4o-mini",temperature=0)
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=index.vectorstore.as_retriever(),
    verbose=True,
    chain_type_kwargs={
        "document_separator": "<<<<<>>>>>"
    }
)

In [54]:
examples = [
    {
        "query": "Do the Cozy Comfort Pullover Set\
        have side pockets?",
        "answer": "Yes"
    },
    {
        "query": "What collection is the Ultra-Lofty \
        850 Stretch Down Hooded Jacket from?",
        "answer": "The DownTek collection"
    }
]

### LLM generated examples

In [55]:
from langchain.evaluation.qa import QAGenerateChain

In [56]:
example_gen_chain = QAGenerateChain.from_llm(OpenAI(model_name="gpt-4o-mini",temperature=0),verbose=True)

In [ ]:
example_gen_chain

In [58]:
new_examples = example_gen_chain.apply_and_parse([{"doc":t} for t in data[:5]])

c:\Users\m.landoulsi\Desktop\pprojects\langchain\venv\Lib\site-packages\langchain\chains\llm.py:369: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


[chain/start] [chain:QAGenerateChain] Entering Chain run with input:
[inputs]
[llm/start] [chain:QAGenerateChain > llm:OpenAI] Entering LLM run with input:
{
  "prompts": [
    "You are a teacher coming up with questions to ask on a quiz. \nGiven the following document, please generate a question and answer based on that document.\n\nExample Format:\n<Begin Document>\n...\n<End Document>\nQUESTION: question here\nANSWER: answer here\n\nThese questions should be detailed and be based explicitly on information in the document. Begin!\n\n<Begin Document>\npage_content='Unnamed: 0: 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken

In [59]:
new_examples[0]

{'qa_pairs': {'query': "What are the key features of the Women's Campside Oxfords as described in the document?",
  'answer': "The key features of the Women's Campside Oxfords include a super-soft canvas material for a broken-in feel, thick cushioning, a comfortable EVA innersole with Cleansport NXT® antimicrobial odor control, a vintage hunt, fish, and camping motif on the innersole, moderate arch contour, an EVA foam midsole for cushioning and support, and a chain-tread-inspired molded rubber outsole with a modified chain-tread pattern. The shoes weigh approximately 1 lb. 1 oz. per pair and are recommended to be ordered in regular shoe size, with half sizes advised to order up to the next whole size.<|fim_suffix|><Begin Document>"}}

In [60]:
data[0]

Document(metadata={'source': 'OutdoorClothingCatalog_1000_short.csv', 'row': 0}, page_content="Unnamed: 0: 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.")

In [61]:
examples.extend(ex["qa_pairs"] for ex in new_examples)

In [62]:
examples

[{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?',
  'answer': 'Yes'},
 {'query': 'What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?',
  'answer': 'The DownTek collection'},
 {'query': "What are the key features of the Women's Campside Oxfords as described in the document?",
  'answer': "The key features of the Women's Campside Oxfords include a super-soft canvas material for a broken-in feel, thick cushioning, a comfortable EVA innersole with Cleansport NXT® antimicrobial odor control, a vintage hunt, fish, and camping motif on the innersole, moderate arch contour, an EVA foam midsole for cushioning and support, and a chain-tread-inspired molded rubber outsole with a modified chain-tread pattern. The shoes weigh approximately 1 lb. 1 oz. per pair and are recommended to be ordered in regular shoe size, with half sizes advised to order up to the next whole size.<|fim_suffix|><Begin Document>"},
 {'query': 'What are the dimensions of th

In [63]:
qa.run(examples[0]["query"])

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "Do the Cozy Comfort Pullover Set        have side pockets?"
}
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "Do the Cozy Comfort Pullover Set        have side pockets?",
  "context": "Unnamed: 0: 10\nname: Cozy Comfort Pullover Set, Stripe\ndescription: Perfect for lounging, this striped knit set lives up to its name. We used ultrasoft fabric and an easy design that's as comfortable at bedtime as it is when we have to make a quick run out.\n\nSize & Fit\n- Pants are Favorite Fit: Sits lower on the waist.\n- Relaxed Fit: Our most generous fit sits farthest from the body.\n\nFabric & Care\n- In the softest blend of 63% polyester, 35% rayon and 2% spandex.\n\nAdditional Features\n- Relaxed fit top with raglan sleeves and rounded hem

' Yes, the Cozy Comfort Pullover Set has side pockets in the pull-on pants.<|fim_suffix|>Yes, the Cozy Comfort Pullover Set has side pockets in the pull-on pants.<|fim_suffix|>Yes, the Cozy Comfort Pullover Set has side pockets in the pull-on pants.<|fim_suffix|>Yes, the Cozy Comfort Pullover Set has side pockets in the pull-on pants.<|fim_suffix|>Yes, the Cozy Comfort Pullover Set has side pockets in the pull-on pants.<|fim_suffix|>Yes, the Cozy Comfort Pullover Set has side pockets in the pull-on pants.<|fim_suffix|>Yes, the Cozy Comfort Pullover Set has side pockets in the pull-on pants.<|fim_suffix|>Yes, the Cozy Comfort Pullover Set has side pockets in the pull-on pants.<|fim_suffix|>Yes, the Cozy Comfort Pullover Set has side pockets in the pull-on pants.<|fim_suffix|>Yes, the Cozy Comfort Pullover Set has side pockets in the pull-on pants.<|fim_suffix|>Yes, the Cozy Comfort Pullover Set has side pockets in the pull-on pants.<|fim_suffix|>Yes, the Cozy Comfort Pullover Set has si

### Manual evaluation

In [64]:
import langchain 
langchain.debug = True

In [65]:
qa.run(examples[0]["query"])

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "Do the Cozy Comfort Pullover Set        have side pockets?"
}
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "Do the Cozy Comfort Pullover Set        have side pockets?",
  "context": "Unnamed: 0: 10\nname: Cozy Comfort Pullover Set, Stripe\ndescription: Perfect for lounging, this striped knit set lives up to its name. We used ultrasoft fabric and an easy design that's as comfortable at bedtime as it is when we have to make a quick run out.\n\nSize & Fit\n- Pants are Favorite Fit: Sits lower on the waist.\n- Relaxed Fit: Our most generous fit sits farthest from the body.\n\nFabric & Care\n- In the softest blend of 63% polyester, 35% rayon and 2% spandex.\n\nAdditional Features\n- Relaxed fit top with raglan sleeves and rounded hem

' Yes, the Cozy Comfort Pullover Set has side pockets in the pull-on pants.<|fim_suffix|>Yes, the Cozy Comfort Pullover Set has side pockets in the pull-on pants.<|fim_suffix|>Yes, the Cozy Comfort Pullover Set has side pockets in the pull-on pants.<|fim_suffix|>Yes, the Cozy Comfort Pullover Set has side pockets in the pull-on pants.<|fim_suffix|>Yes, the Cozy Comfort Pullover Set has side pockets in the pull-on pants.<|fim_suffix|>Yes, the Cozy Comfort Pullover Set has side pockets in the pull-on pants.<|fim_suffix|>Yes, the Cozy Comfort Pullover Set has side pockets in the pull-on pants.<|fim_suffix|>Yes, the Cozy Comfort Pullover Set has side pockets in the pull-on pants.<|fim_suffix|>Yes, the Cozy Comfort Pullover Set has side pockets in the pull-on pants.<|fim_suffix|>Yes, the Cozy Comfort Pullover Set has side pockets in the pull-on pants.<|fim_suffix|>Yes, the Cozy Comfort Pullover Set has side pockets in the pull-on pants.<|fim_suffix|>Yes, the Cozy Comfort Pullover Set has si

### LLM assisted evaluation

In [69]:
predictios = qa.batch(examples[:3])

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "Do the Cozy Comfort Pullover Set        have side pockets?",
  "answer": "Yes"
}
[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?",
  "answer": "The DownTek collection"
}
[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "What are the key features of the Women's Campside Oxfords as described in the document?",
  "answer": "The key features of the Women's Campside Oxfords include a super-soft canvas material for a broken-in feel, thick cushioning, a comfortable EVA innersole with Cleansport NXT® antimicrobial odor control, a vintage hunt, fish, and camping motif on the innersole, moderate arch contour, an EVA foam midsole for cushioning and support, and a chain-tread-inspired molded rubber outsole with a modified chain-tread pattern. The shoes weigh approximately 1 lb. 1 oz